In [14]:
# Collate data from parametric sweeps for combined analysis plots
import numpy as np
from scipy import signal
from scipy import interpolate
import matplotlib.pyplot as plt 
import os
import pyfilaments.analysisutils as analysis
from tqdm import tqdm
import pandas as pd

from joblib import Parallel, delayed
import multiprocessing

# Folder containing data
data_folder = '/home/deepak/LacryModelling_Local/SimulationData_ForAnalysis/2021-02-06'

df_dict = {}
# Find all simulation data files and create a list
files_list = []
 # Walk through the folders and identify the simulation data files
for dirs, subdirs, files in os.walk(data_folder, topdown=False):
   
    root, subFolderName = os.path.split(dirs)
 
    if('SearchCoverage' in subFolderName):
        
        root, *rest = os.path.split(root)
   
        # Get the metadata for this condition
        df_metadata = pd.read_csv(os.path.join(root, 'metadata.csv'))
        print(df_metadata['spring constant'][0])
        df_dict[df_metadata['spring constant'][0]] = pd.DataFrame({})
        
        for file in files:
            if('timeseries' in file):
                
                df_current = pd.read_csv(os.path.join(dirs,file))
                df_dict[df_metadata['spring constant'][0]] = df_dict[df_metadata['spring constant'][0]].append(df_current)



    
# # for file in tqdm(files_list):
	
# def run_filament_analysis(file):
# 	print('Analyzing file ...')
# 	print(file)

# 	filament = analysis.analysisTools(file = file)
# 	filament.filament_tip_coverage(save = True)


# num_cores = multiprocessing.cpu_count()

# num_cores = 12

# results = Parallel(n_jobs=num_cores,  verbose=10)(delayed(run_filament_analysis)(file) for file in tqdm(files_list))


50
40
20
70
100
30


In [16]:
df_dict[50]

,Unnamed: 0,Time,Unique positions count
0,0,0.0,1.0
1,1,10.0,1.0
2,2,20.0,1.0
3,3,30.0,1.0
4,4,40.0,1.0
...,...,...,...
99996,99996,999960.0,436.0
99997,99997,999970.0,436.0
99998,99998,999980.0,436.0
99999,99999,999990.0,436.0


In [ ]:
import seaborn as sns
# Plot the timer-series of unique locations sampled
plt.style.use("dark_background")

plt.figure()
for key in df_dict.keys():
    
    sns.lineplot()
    
    